<a href="https://colab.research.google.com/github/aliabdelmonam/Poisonous_Mushrooms/blob/main/Poisonous_Mushrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e8:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F76727%2F9045607%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240829%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240829T122852Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3435ec23338b9b21a3d9b106ef3005978285be6c9b2ed95e7568ccff573f91c0f059eb27aef85ff90498a66cb8b5cf9f5073503cd4c7820c1beeda66dbfa44b5ba8e98b909806d2d711d33dc1369bbde33ab6f623f18a5a483d155c15e8985c110eaa63b756b5293e97f144f3df3e290eb11e52bc426c3c00828b79187da3e1e9476a26ebe44ea54d6504c0aa2835d949d5ca99a0ff56647982dfba89ad38474c45ba245e026670ad1ec3005f2675431fc002edfe315c0a76d7134e2879fc8d98448265ba679142e14fe0bfe04775b6292cd6569555a70be44fa897562f330b8d8dfccb14008a69591b18e773a2916015d3e14a6bf1013619d5038c94419c34a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


### **Libraries**

In [ ]:
# !pip install scikit-optimize
# !pip install catboost
# !pip install prince


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import copy
# import prince


## Data

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# df=pd.read_csv('/content/sample_submission.csv')
tr=pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")


In [ ]:
tr.shape

In [ ]:
tr.head()

In [ ]:
tr.info()

In [ ]:
df_train = copy.deepcopy(tr)

# **Category**

In [ ]:
def subclass_details (df):
  df=df.select_dtypes(include='category')
  for col in df.columns:
    print('Number of unique classes:',df[col].nunique())
    print(df[col].value_counts().head(10))
    print('############################')

In [ ]:
def remove_att(df,threshold=200):
  cat_coln=df.select_dtypes(include='object')
  for col in cat_coln:
    attrib_drop=[]
    for att , count in df[col].value_counts().items() :
      if count <threshold:
        attrib_drop.append(att)
    mask = df[col].isin(attrib_drop)
    df.loc[mask,col] = 'UNK'
  return df

In [ ]:
def convert_cate (df):
  for clas in df.select_dtypes(include='object'):
    df[clas] =   df[clas].astype('category')
  return df

In [ ]:
df_train = remove_att(df_train)
df_train = convert_cate(df_train)
subclass_details(df_train)

In [ ]:
df_train.info()

# **Missing Value**

In [ ]:
def plot_missing_feature(df):
  null_df=(df.isna().sum()*100/df.shape[0]).sort_values(ascending=False)
  sns.barplot(x=null_df.index,y=null_df.values,palette='plasma')
  plt.xticks(rotation=90)
  plt.xlabel('Feature')
  plt.ylabel('Percent(%)')
  plt.title('Missing Values')
  plt.show()

In [ ]:
def missing_feature (df):
  null_df=(df.isna().sum()*100/df.shape[0]).sort_values(ascending=False)
  return null_df

In [ ]:
null_df_train = missing_feature(df_train)
null_df_train

In [ ]:
plot_missing_feature(df_train)

In [ ]:
def columns_drop(df):
  column_drop=[]
  null_df=missing_feature(df)
  for col,val in null_df.items():
    if val >4:
      column_drop.append(col)
  return column_drop

In [ ]:
column_drop_train = columns_drop(df_train)
print('-----------------------------------------')
print(column_drop_train)

In [ ]:
df_train.drop(column_drop_train,axis=1,inplace=True)
df_train.drop('id',axis=1,inplace=True)

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.plot(kind='hist',subplots=True,sharex=True,figsize=(15,15),bins=100)

# **Splitting Data**

In [ ]:
x=df_train.drop('class',axis=1)
y=df_train['class']
y=np.array([0 if i =='e' else 1 for i in y])
y.reshape(-1,1)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=.7,stratify=y)

# **Pipeline**

In [ ]:
num_data_train_columns = x.select_dtypes(include='number').columns
cat_data_train_columns = x.select_dtypes(include='category').columns
cat_data_train_columns

In [ ]:
num_pipe=Pipeline (steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])
cat_pipe=Pipeline (steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(sparse=False, handle_unknown='ignore'))
    # ('encoder',OrdinalEncoder())

])

In [ ]:
df_preprocessing=ColumnTransformer(
     transformers=[
        ('num', num_pipe, num_data_train_columns),
        ('cat', cat_pipe, cat_data_train_columns)
    ]
)

In [ ]:
final_pipe = Pipeline(steps=[
    ('preprocessor',df_preprocessing),
    ('PCA',PCA(n_components=.95)),
#     ('MCA',prince.MCA( n_components=2,  n_iter=3,       check_input=True, engine='auto',   random_state=42))

])

In [ ]:
x_train=final_pipe.fit_transform(x_train)
x_test=final_pipe.transform(x_test)

# **Modeling**

In [ ]:
def Bayesian_Optimization (model,search_space):
  bayes = BayesSearchCV(model,
                        search_space,
                         n_iter= 10,
                        n_jobs=-1,
                        scoring='accuracy',
                        random_state=42)
  return bayes

In [ ]:
xgb_space = {
    'n_estimators': Integer(50, 150),
    'max_depth': Integer(2, 8),
    'learning_rate': Real(0.01, .4, 'log-uniform'),
    'subsample': Real(0.5, 1.0, 'uniform'),
    'colsample_bytree': Real(0.5, 1.0, 'uniform'),
    'alpha':Real(.1,.5,'uniform'),
    'min_child_weight':Integer(5,10)
}
lgb_space={'num_leaves': Integer(24, 45),
          'feature_fraction': Real(0.1, 0.9),
          'bagging_fraction': Real(0.5, 1),
          'max_depth':Integer (5, 9),
          'lambda_l1':Real (0, 5),
          'lambda_l2':Real (0, 3),
          'min_split_gain':Real (0.001, 0.1),
          'min_child_weight': Integer(5, 60)
}
cat_space={
    'iterations': Integer(10, 100),
    'depth': Integer(1, 8),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'l2_leaf_reg': Real(1e-3, 10, 'log-uniform'),
    'border_count': Integer(32, 128)
}

In [ ]:
clsss=len(y[y==0]) / len(y[y==1])
clsss

In [ ]:
xgb = XGBClassifier(random_state=42,scale_pos_weight =clsss)
# lgb = LGBMClassifier(random_state=42)
cat = CatBoostClassifier(random_state=42)
xgb_optimzied = Bayesian_Optimization(xgb,xgb_space)
# lgb_optimzied = Bayesian_Optimization(lgb,lgb_space)
cat_optimzied = Bayesian_Optimization(cat,cat_space)

In [ ]:
xgb_optimzied.fit(x_train,y_train)
# lgb_optimzied.fit(x_train,y_train)
cat_optimzied.fit(x_train,y_train)

In [ ]:
xgb_optimzied.best_params_

In [ ]:
xgb=xgb_optimzied.best_estimator_
# lgb=lgb_optimzied.best_estimator_
cat=cat_optimzied.best_estimator_

In [ ]:
y_pred_xgb=xgb.predict(x_test)
# y_pred_lgm=lgb.predict(x_test)
y_pred_cat=cat.predict(x_test)
print('XGB --> ',accuracy_score(y_test,y_pred_xgb))
# print('LGM --> ',accuracy_score(y_test,y_pred_lgm))
print('CAT --> ',accuracy_score(y_test,y_pred_cat))

In [ ]:
voting_clf=VotingClassifier(estimators=[
        ('xgb', xgb),
        ('catboost', cat)
#         ,('lightbgm',lbm)
    ],
    voting='soft'
)
voting_clf.fit(x_train,y_train)
y_pred_voting=voting_clf.predict(x_test)
print('Voting Classifier Accuracy',accuracy_score(y_test,y_pred_voting))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_voting))

In [ ]:
import gc
gc.collect()

> ***Submession***

In [ ]:
ts = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')

In [ ]:
df_test = copy.deepcopy(ts)
df_test.drop('id',axis=1,inplace=True)

In [ ]:
columns_drop(ts)

In [ ]:
df_test=final_pipe.transform(df_test)

In [ ]:
final_pred = voting_clf.predict(df_test)

In [ ]:
final_pred_trans=['e' if i==0 else 'p' for i in final_pred]

In [ ]:
submession= pd.DataFrame({'id':ts['id'].values,
                          'class':final_pred_trans
                         }
                        )


In [ ]:
submession.to_csv('submession.csv',index=False)